In [1]:
from ipysigma import Sigma
import networkx as nx
import csv

In [2]:
with open('../data/fosdem-tracks-pseudo.csv') as f:
    data = list(csv.DictReader(f))

In [3]:
for key, value in data[0].items():
    if value:
        print(key, '=', value)

year = 2007
event_id = 12
track = openingclosingtalks
title = Welcome
speakers_name = Nial Keiona


In [14]:
graph = nx.Graph()

for row in data:
    if not row['speakers_name']:
        continue
        
    track = row['track']
    year = row['year']
    
    graph.add_node(track, part='track')
    
    for speaker in row['speakers_name'].split('|'):
        graph.add_node(speaker, part='speaker')
        
        # Keeping info about last pair (most recent)
        if not graph.has_edge(speaker, track):
            graph.add_edge(speaker, track, count=1, year=year)
        else:
            graph[speaker][track]['count'] += 1
            graph[speaker][track]['year'] = year

In [15]:
graph.order(), graph.size()

(5108, 6210)

In [16]:
Sigma(graph, node_size=graph.degree, node_color='part', edge_size='count', node_border_color_from='node', node_size_range=(2, 20))

Sigma(nx.Graph with 5,108 nodes and 6,210 edges)

In [46]:
YEAR_2010 = lambda n: sum(1 if a['year'] == '2012' else 0 for _s, _t, a in graph.edges(n, data=True))
YEAR_2015 = lambda n: sum(1 if a['year'] == '2022' else 0 for _s, _t, a in graph.edges(n, data=True))

In [47]:
from ipysigma import SigmaGrid

In [48]:
grid = SigmaGrid(graph, node_size=graph.degree, node_halo_color_gradient='OrRd', default_node_color='rgba(0, 0, 0, 0.0)')
grid.add(node_halo_size=YEAR_2010, node_halo_color=YEAR_2010, node_zindex=YEAR_2010)
grid.add(node_halo_size=YEAR_2015, node_halo_color=YEAR_2015, node_zindex=YEAR_2015)
grid

In [7]:
!pip install pelote

In [8]:
from pelote import monopartite_projection, crop_to_largest_connected_component

In [9]:
for node, attr in graph.nodes.data():
    attr['bipartite_degree'] = graph.degree(node)

In [10]:
mono = monopartite_projection(graph, 'track', metric='overlap')

In [11]:
crop_to_largest_connected_component(mono)

In [12]:
Sigma(mono, node_size="bipartite_degree", node_metrics=['louvain'], node_color='louvain', edge_size='weight')

Sigma(nx.Graph with 176 nodes and 1,283 edges)